In [1]:
!git clone https://github.com/theschoolofai/YoloV3.git

Cloning into 'YoloV3'...
remote: Enumerating objects: 156, done.
remote: Total 156 (delta 0), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (156/156), 12.74 MiB | 31.67 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Youtube Video

In [6]:
# !pip install youtube-dl
# !youtube-dl https://www.youtube.com/watch?v=5Mts9GGv3gk
# !ffmpeg -ss 00:00:00 -i video.mp4 -to 00:02:00 -c copy cut.mp4
!ffmpeg -i cut.mp4 -vf fps=23.9 YoloV3/video_frames/%d.jpg

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [8]:
# Download weights 
# https://drive.google.com/file/d/1UcR-zVoMs7DH5dj3N1bswkiQTA4dmKF4/view?usp=sharing
# !gdown https://drive.google.com/uc?id=1UcR-zVoMs7DH5dj3N1bswkiQTA4dmKF4
# !mkdir YoloV3/weights
# !mv yolov3-spp-ultralytics.pt YoloV3/weights/

In [11]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.8.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
ls

gdrive/  sample_data/  YoloV3/


In [9]:
cd YoloV3/

/content/YoloV3


In [10]:
!ls

 annotation_tool   __pycache__	     test.py	        video_frames
 cfg		   README.md	     ting	        vim.exe.stackdump
 data		   results.json      train_batch0.png   weights
 detect.py	   results.png	     train.py
 models.py	   results.txt	    'ubdivisions=1'
 output		   test_batch0.png   utils


In [17]:
# !wget https://transfer.sh/gb9lJ/data.zip
!unzip data.zip -d zoheb_data

Streaming output truncated to the last 5000 lines.
  inflating: zoheb_data/data/assignment13/labels/Himage_052.txt  
  inflating: zoheb_data/__MACOSX/data/assignment13/labels/._Himage_052.txt  
  inflating: zoheb_data/data/assignment13/labels/Mimg_108.txt  
  inflating: zoheb_data/__MACOSX/data/assignment13/labels/._Mimg_108.txt  
  inflating: zoheb_data/data/assignment13/labels/z037 .txt  
  inflating: zoheb_data/__MACOSX/data/assignment13/labels/._z037 .txt  
  inflating: zoheb_data/data/assignment13/labels/images (52).txt  
  inflating: zoheb_data/__MACOSX/data/assignment13/labels/._images (52).txt  
  inflating: zoheb_data/data/assignment13/labels/Bimg_5.txt  
  inflating: zoheb_data/__MACOSX/data/assignment13/labels/._Bimg_5.txt  
  inflating: zoheb_data/data/assignment13/labels/images (1).txt  
  inflating: zoheb_data/__MACOSX/data/assignment13/labels/._images (1).txt  
  inflating: zoheb_data/data/assignment13/labels/Himage_085.txt  
  inflating: zoheb_data/__MACOSX/data/assignm

In [18]:
!cp -r zoheb_data/data/assignment13 data/

In [21]:
# !rm cfg/yolov3-custom.cfg
# !wget "https://transfer.sh/pt0Q7/yolov3-custom.cfg" -O cfg/yolov3-custom.cfg

--2021-03-20 14:36:01--  https://transfer.sh/pt0Q7/yolov3-custom.cfg
Resolving transfer.sh (transfer.sh)... 144.76.136.153
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8592 (8.4K) []
Saving to: ‘cfg/yolov3-custom.cfg’

cfg/yolov3-custom.c 100%[===================>]   8.39K  --.-KB/s    in 0.001s  

2021-03-20 14:36:04 (11.8 MB/s) - ‘cfg/yolov3-custom.cfg’ saved [8592/8592]



In [19]:
!python train.py --data data/smalcoco/smalcoco.data --batch 10 --cache --epochs 3 --nosave

Namespace(accumulate=4, adam=False, batch_size=10, bucket='', cache_images=True, cfg='cfg/yolov3-spp.cfg', data='data/smalcoco/smalcoco.data', device='', epochs=3, evolve=False, img_size=[512], multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/yolov3-spp-ultralytics.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15109MB)

2021-03-20 14:33:33.382028: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Model Summary: 225 layers, 6.29987e+07 parameters, 6.29987e+07 gradients
Caching labels (20 found, 0 missing, 0 empty, 0 duplicate, for 20 images): 100% 20/20 [00:00<00:00, 2697.04it/s]
Caching images (0.0GB): 100% 20/20 [00:00<00:00, 113.46it/s]
Caching labels (20 found, 0 missing, 0 empty, 0 duplicate, for 20 images): 100% 20/20 [00:00<00:00, 6153.16it/

In [22]:
!python train.py --data data/assignment13/custom.data --batch 10 --cache --cfg cfg/yolov3-custom.cfg --epochs 5 --nosave

Namespace(accumulate=4, adam=False, batch_size=10, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/assignment13/custom.data', device='', epochs=5, evolve=False, img_size=[512], multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/yolov3-spp-ultralytics.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15109MB)

2021-03-20 14:37:20.461102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Caching labels (3034 found, 131 missing, 38 empty, 0 duplicate, for 3203 images): 100% 3203/3203 [00:00<00:00, 7697.37it/s]
Caching images (1.8GB): 100% 3203/3203 [00:19<00:00, 167.36it/s]
Caching labels (297 found, 14 missing, 7 empty, 0 duplicate, for 318 images): 100% 318/318

In [25]:
# !mkdir video_output
!python detect.py --source video_frames --names data/assignment13/custom.names --conf-thres 0.1 --output video_output

Namespace(agnostic_nms=False, augment=False, cfg='cfg/yolov3-custom.cfg', classes=None, conf_thres=0.1, device='', fourcc='mp4v', half=False, img_size=512, iou_thres=0.6, names='data/assignment13/custom.names', output='video_output', save_txt=False, source='video_frames', view_img=False, weights='weights/last.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15109MB)

Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
image 1/2869 video_frames/1.jpg: 320x512 2 hardhats, 1 vests, Done. (0.033s)
image 2/2869 video_frames/10.jpg: 320x512 1 hardhats, 1 vests, Done. (0.026s)
image 3/2869 video_frames/100.jpg: 320x512 1 hardhats, 1 vests, Done. (0.026s)
image 4/2869 video_frames/1000.jpg: 320x512 Done. (0.026s)
image 5/2869 video_frames/1001.jpg: 320x512 Done. (0.026s)
image 6/2869 video_frames/1002.jpg: 320x512 Done. (0.026s)
image 7/2869 video_frames/1003.jpg: 320x512 Done. (0.026s)
image 8/2869 video_frames/1004.jpg: 320x512 Done. (0.026s)
i

In [26]:
!ffmpeg -framerate 23.9 -i video_output/%d.jpg final.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib